<b>Quiz 3 - Problem 1</b>

## Offline Section

### Linear Formulation

The problem resembles a **transshipment problem** the most. In a transshipment problem, commodities can be transported directly from sources to destinations, and can also be shipped indirectly via one or more transshipment points where the commodity can change its mode of transportation. In this case, the beer can be shipped directly from factories to retail points or indirectly through warehouses.

### Decision Variables:
Let $X_{ij}$ denote the amount of beer shipped from node $i$ to node $j$.

### Objective:
We want to minimize the total shipping cost. So, the objective function is:
$$\min \sum_{i \in \text{{Nodes}}} \sum_{j \in \text{{Nodes}}} C_{ij}X_{ij}$$

### Constraints:

Flow conservation:
For each node $i$ in Nodes, the inflow to node $i$ should be equal to the outflow from node $i$ plus its supply/demand:

$$
\sum_{j \in \text{{Nodes}}} X_{ji} - \sum_{j \in \text{{Nodes}}} X_{ij} = S_i, \forall i \in \text{{Nodes}}
$$

Here, $S_i$ is the supply (if negative) or demand (if positive) at node $i$. 

Non-negativity:
$$
X_{ij} \ge 0, \forall (i,j) \in \text{{Arcs}}
$$


### Python

In [1]:
# data

import csv
f = open("quiz 3 problem 1.csv")
csvfile = csv.DictReader(f, delimiter=',')
headers = csvfile.fieldnames

table = []
for row in csvfile:
    table.append(row)
    
f.close()

# Create set of nodes
Nodes = set()
# start by adding the elements in the header (except for first and last element)
for i in range(1,len(headers)-1):
    Nodes.add(headers[i])

# Create dictionaries "Cost" and "Supply" (for the retail nodes)
# We can borrow the keys from "Cost" to define the arc set of the network.
Cost = {}
Capacity = {}
Supply = {}
for row in table:
    val = row['from']
    if val != 'Demand':
        for i in Nodes:
            if (row[i] != '-'):
                Cost[val,i] = float(row[i])
    else:
        for i in Nodes:
            if (row[i] != '-'):
                Supply[i] = float(row[i])

# Add the "Supply" data for the factories and complete the node set.
for row in table:
    val = row['from']
    if val != 'Demand':
        if row['Capacity'] != '-':
            Supply[val] = -float(row['Capacity'])
        Nodes.add(val)

# Complete the "Supply" data for the nodes without a supply (i.e., the 
# warehouses have 0 demand/supply).
for i in Nodes:
    if not(i in Supply.keys()):
        Supply[i] = 0
    
# Define the arc set
Arcs = Cost.keys()

Capacity = {'F1': 3200.0, 'F2': 2500.0} #This is capture in Supply but just for documentation purposes

In [2]:
Nodes

{'F1', 'F2', 'R1', 'R2', 'R3', 'R4', 'W1', 'W2'}

In [3]:
Cost

{('F1', 'R1'): 0.8,
 ('F1', 'W1'): 0.5,
 ('F1', 'W2'): 0.64,
 ('F2', 'R4'): 1.17,
 ('F2', 'W1'): 0.4,
 ('F2', 'W2'): 0.55,
 ('W1', 'R3'): 0.49,
 ('W1', 'R1'): 0.23,
 ('W1', 'R2'): 0.45,
 ('W2', 'R4'): 0.64,
 ('W2', 'R3'): 0.25,
 ('W2', 'R2'): 0.85}

In [4]:
Capacity

{'F1': 3200.0, 'F2': 2500.0}

In [5]:
Supply

{'R4': 900.0,
 'R3': 1100.0,
 'R1': 700.0,
 'R2': 1800.0,
 'F1': -3200.0,
 'F2': -2500.0,
 'W1': 0,
 'W2': 0}

In [7]:
from docplex.mp.model import Model
mdl = Model()

# Variables
shipped = mdl.continuous_var_dict(Arcs, lb=0, name='shipped')

# Objective
mdl.minimize(mdl.sum(Cost[i, j] * shipped[i, j] for i, j in Arcs))

# Flow conservation constraints
for j in Nodes:
    inflow = mdl.sum(shipped[i, j] for i in Nodes if (i, j) in Arcs)
    outflow = mdl.sum(shipped[j, i] for i in Nodes if (j, i) in Arcs)
    mdl.add_constraint(inflow - outflow == Supply[j])

In [8]:
# solve
mdl.solve()
mdl.get_solve_details()

docplex.mp.SolveDetails(time=0.015,status='infeasible')

In [ ]:
mdl.print_solution()

## Quiz Section